<a href="https://colab.research.google.com/github/Nataliaricotorres/ean_PNL/blob/main/Tarea_de_modelos_de_conteo_%5BCountVectorizer%5D_%5BTfidfVectorizer%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea de modelos de conteo


---
INTEGRANTES
*  Geraldine Patiño
*  Natalia Rico
*  



